In [2]:
import boto3
import sagemaker
import time
from time import strftime

boto_session = boto3.Session()
sagemaker_session = sagemaker.Session(boto_session=boto_session)
sm_client = boto3.client("sagemaker")
region = boto_session.region_name
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()
account = sagemaker_session.boto_session.client("sts").get_caller_identity()["Account"]

prefix = 'sagemaker-intel-dvc'

print(f"account: {account}")
print(f"bucket: {bucket}")
print(f"region: {region}")
print(f"role: {role}")

account: 546801796757
bucket: sagemaker-ap-south-1-546801796757
region: ap-south-1
role: arn:aws:iam::546801796757:role/sagemaker_manual_role


In [3]:
from sagemaker.pytorch import PyTorchPredictor
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

In [4]:
predictor_new = PyTorchPredictor(
    endpoint_name="last-assignment-prod",
    sagemaker_session=sagemaker_session,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
)

In [5]:
import torch

import torchvision.transforms as T
import torch.nn.functional as F

from PIL import Image

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
transforms = T.Compose([
    T.ToTensor(),
    T.Resize((224, 224)),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [7]:
classnames = ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']


In [8]:
import requests
import numpy as np
from PIL import Image
from io import BytesIO

# image 1 - Forest
url = 'https://media.istockphoto.com/id/901134626/photo/sun-shining-in-a-forest.jpg?s=612x612&w=0&k=20&c=Z1GPER9EE6BQK_efVKaSO7WODScQYOMPX3TFFPCZt88='
response = requests.get(url)
im = Image.open(BytesIO(response.content))
im = transforms(im)
dummy_data = {"inputs": im[None, ...].numpy().tolist()}
f"Prediction: {classnames[torch.argmax(torch.tensor(predictor_new.predict(dummy_data)), dim=-1)[0]]}"

'Prediction: forest'

In [9]:
import requests
import numpy as np
from PIL import Image
from io import BytesIO

# image 1 - Glacier
url = 'https://media.istockphoto.com/id/1061549950/photo/calving-glacier-alaska-hubbard-glacier-a-huge-iceberg-calves-into-disenchantment-bay-st-elias.jpg?s=612x612&w=0&k=20&c=ri7A1uiTq5PI5iSxzmt22oAAOtrizXffP4JvA6nFTf4='
response = requests.get(url)
im = Image.open(BytesIO(response.content))
im = transforms(im)
dummy_data = {"inputs": im[None, ...].numpy().tolist()}
f"Prediction: {classnames[torch.argmax(torch.tensor(predictor_new.predict(dummy_data)), dim=-1)[0]]}"

'Prediction: glacier'